In [5]:
import pickle
import time
from typing import List

import numpy as np
from tqdm import tqdm
import plotly.graph_objects as go
import plotly.express as px

from quara.objects.composite_system import CompositeSystem
from quara.objects.elemental_system import ElementalSystem
from quara.objects.matrix_basis import get_normalized_pauli_basis, get_pauli_basis, get_comp_basis
from quara.objects.povm import (
    Povm,
    get_x_measurement,
    get_y_measurement,
    get_z_measurement,
)
from quara.objects.qoperation import QOperation
from quara.objects.state import State, get_x0_1q,  get_x1_1q, get_y0_1q, get_y1_1q, get_z0_1q, get_z1_1q
from quara.protocol.qtomography.standard.standard_qst import StandardQst
from quara.protocol.qtomography.standard.standard_povmt import StandardPovmt
from quara.protocol.qtomography.standard.linear_estimator import LinearEstimator

In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
def estimate(name: str,
             true_object: Povm,
             states: List[State],
             measurement_n: int,
             num_data: List[int], iterations: int, on_para_eq_constraint: bool=True):
    povmt = StandardPovmt(states, measurement_n, on_para_eq_constraint=on_para_eq_constraint)

    # generate empi dists and calc estimate
    obj_sequences = []
    start = time.time()
    for iteration in range(iterations):
        seeds = [iteration] * len(num_data)
        empi_dists_seq = povmt.generate_empi_dists_sequence(
            true_object, num_data, seeds
        )

        estimator = LinearEstimator()
        obj_sequence = estimator.calc_estimate_sequence(povmt, empi_dists_seq)

        info = {
            "iteration": iteration,
            "empi_dists_seq": empi_dists_seq,
            "obj_sequence": obj_sequence.estimated_var_sequence,
        }
        print(info)
        obj_sequences.append(obj_sequence.estimated_qoperation_sequence)

    end = time.time()
    print(f"time(s)={end - start}")
    #with open(f"state_obj_sequences_{name}_{on_para_eq_constraint}_{iterations}.pkl", "wb") as f:
    #    pickle.dump(obj_sequences, f)

    #with open("obj_sequences.pkl", "rb") as f:
    #    obj_sequences = pickle.load(f)
    return obj_sequences

In [8]:
# setup system
e_sys = ElementalSystem(0, get_normalized_pauli_basis())
c_sys = CompositeSystem([e_sys])

# |+><+|
state_x0 = get_x0_1q(c_sys)
# |+i><+i|
state_y0 = get_y0_1q(c_sys)
# |0><0|
state_z0 = get_z0_1q(c_sys)
# |1><1|
state_z1 = get_z1_1q(c_sys)
states = [state_x0, state_y0, state_z0, state_z1]

In [9]:
# parameter
num_data = [1000]
iterations = 10
true_object = get_z_measurement(c_sys)
measurement_n = 2  # 測定値の数

param_affine_est_linear = estimate("z0", true_object, states, measurement_n, num_data, iterations, on_para_eq_constraint=True)
param_linear_est_linear = estimate("z1", true_object, states, measurement_n, num_data, iterations, on_para_eq_constraint=False)

TypeError: generate_empi_dists_sequence() takes 2 positional arguments but 3 were given

# Physicality Violation

## on_para_eq_constraint = True

In [9]:
param_affine_est_linear

[[<quara.objects.povm.Povm at 0x11542b940>],

In [11]:
value_list = []

for povm_est in param_affine_est_linear:
    povm_est = povm_est[0]  # TODO
    # tr = np.trace(state_est.to_density_matrix())
    # value = tr.real
    # value_list.append(value)